In [1]:
import pandas as pd

In [257]:
df = pd.read_csv("spotify_millsongdata.csv")
df_tracks = pd.read_csv("user_top_tracks.csv")
df_artists = pd.read_csv("user_top_artists.csv")
df_albums = pd.read_csv("user_top_albums.csv")
df.head()

df_tracks.head()

,user_id,rank,track_name,artist_name,playcount,mbid
0,1,1,Ice Cream,New Young Pony Club,54,417a3d0b-0291-4e08-b167-50971dbdae36
1,1,2,Hiding On The Staircase,New Young Pony Club,51,f6ac2fd2-b33b-48ff-a094-537cf530cf20
2,1,3,Descend,New Young Pony Club,49,33434fd1-79a1-3882-bb31-fbd430815a93
3,1,4,Air War,Crystal Castles,48,06d3fea3-e806-35ed-9562-aa396a97de14
4,1,5,Get Lucky,New Young Pony Club,48,ab351b25-7126-3792-9b0d-ce44fd0c0462


In [259]:
df_tracks.tail()

,user_id,rank,track_name,artist_name,playcount,mbid
23822545,479484,46,teeth,Loser,192,NaN
23822546,479484,47,MISA MISA!,Corpse,191,NaN
23822547,479484,48,title track,Machine Gun Kelly,191,b9874074-b2d6-4135-bc2a-f4ef44b0ae69
23822548,479484,49,В последний раз,Tima Belorusskih,190,NaN
23822549,479484,50,fantasize,ericdoa,189,NaN


In [3]:
print(df_tracks.loc[df_tracks["track_name"] == "You Reposted in the Wrong Neighborhood"])

          user_id  rank                              track_name artist_name  \
22254538   447927    39  You Reposted in the Wrong Neighborhood      ＳＨＯＫＫ青   

          playcount mbid  
22254538         68  NaN  


In [4]:
if("link" in df):
    df = df.drop("link", axis = 1).reset_index(drop = True)

In [5]:
df = df.sample(5000)

In [6]:
df["text"] = df["text"].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [7]:
df['text'].iloc[1]

"i took a walk out to the sun but i just, just couldn't take it  \r i followed a dream it was circular  \r but i just, just could not fake it  \r   \r step to the left the left the left step to the right the right the right  \r pick up the sign the sign the sign for a stride of the mind the mind  \r   \r simon of the desert blew into town on the scalding tail of a bright cold wind  \r slipped through the sand footprints emerged  \r where no one was walking simon had been  \r   \r dropped from heaven to a ready made world  \r said i'm no sufi but i'll give it a whirl  \r we booked passage on the book of the dead  \r time to travel simon said  \r   \r step to the left the left the left step to the right the right the right  \r pick up the sign the sign the sign on the stride of the mind the mind the mind  \r   \r come on move where dreams increase where every man is a masterpiece  \r if you want to be counted as another kind  \r and you're true, pursue stride of the mind the mind the min

In [8]:
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def token(str):
    token = nltk.word_tokenize(str)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

token("my beautiful, beauty")

'my beauti , beauti'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfid = TfidfVectorizer(analyzer="word", stop_words="english")
matrix = tfid.fit_transform(df["text"])

In [10]:
similar = cosine_similarity(matrix)
similar[0]

array([1.        , 0.01101588, 0.0715048 , ..., 0.02061275, 0.00147596,
       0.01413213])

In [11]:
test_song = df.loc[df["song"] == "Innocence Lost"]
print(test_song)

def recommender_function(song):
    index = df.loc[df["song"] == song].index[0]
    distance = sorted(list(enumerate(similar[index])), reverse = True, key = lambda x:x[1])
    songs = []
    for id in distance[1:5]:
        songs.append(df.iloc[id[0]].song)
    return songs

def artist_recommender_function(artist):
    index = df.loc[df["artist"] == artist].index[0]
    distance = sorted(list(enumerate(similar[index])), reverse = True, key = lambda x:x[1])
    songs = []
    for id in distance[1:5]:
        songs.append(df.iloc[id[0]].artist)
    return songs

print(artist_recommender_function("Kanye West"))

Empty DataFrame
Columns: [artist, song, text]
Index: []


IndexError: index 39260 is out of bounds for axis 0 with size 5000

In [13]:
print(df.loc[df["artist"] == "ABBA"])

print(top_50_albums(50))

    artist                        song  \
24    ABBA  Gonna Sing You My Lovesong   
37    ABBA              I Have A Dream   
79    ABBA             Rubber Ball Man   
45    ABBA               Juper Jrouper   
110   ABBA   When I Kissed The Teacher   
27    ABBA              Happy New Year   
104   ABBA                 Voulez Vous   
91    ABBA     Thank You For The Music   

                                                  text  
24   you say she's been mad at you  \r then you say...  
37   i have a dream, a song to sing  \r to help me ...  
79   the poster on the wall of a dear friend  \r i ...  
45   super trooper beams are gonna blind me  \r but...  
110  everybody screamed when i kissed the teacher  ...  
27   no more champagne  \r and the fireworks are th...  
104  people everywhere  \r a sense of expectation h...  
91   i'm nothing special, in fact i'm a bit of a bo...  


NameError: name 'top_50_albums' is not defined

In [15]:
def top_50_songs(user):
    temp = []
    for track in df_tracks.loc[df_tracks["user_id"] == user, "track_name"]:
        temp.append(track)
    return temp

def top_50_artists(user):
    temp = []
    for artist in df_artists.loc[df_artists["user_id"] == user, "artist_name"]:
        temp.append(artist)
    return temp

def top_50_albums(user):
    temp = []
    for album in df_albums.loc[df_albums["user_id"] == user, "album_name"]:
        temp.append(album)
    return temp

def user_similarity(user1, user2):
    sum = 0
    user1_songs, user2_songs = top_50_songs(user1), top_50_songs(user2)
    user1_artists, user2_artists = top_50_artists(user1), top_50_artists(user2)
    user1_albums, user2_albums = top_50_albums(user1), top_50_albums(user2)
    similar_songs, similar_artists, similar_albums = [], [], []
    for i in range(50):
        song, artist, album = user1_songs[i], user1_artists[i], user1_albums[i]
        if(song in user2_songs or artist in user2_artists or album in user2_albums):
            sum += 1
            if(song in user2_songs):
                similar_songs.append(song)
            elif(artist in user2_artists):
                similar_artists.append(artist)
            else:
                similar_albums.append(album)
    if(sum > 0):
        print(similar_songs, similar_artists, similar_albums)
    return sum

start = 5
count = 100
threshold = 5

for i in range(start + 1, start + count):
    if(user_similarity(start, i) > threshold):
        print(start, i, user_similarity(start, i))

print(top_50_songs(5), "\n\n", top_50_songs(21))

[] ['Amy Winehouse'] ['Back to Black']
['Time to Pretend', 'Walking on a Dream'] ['Arcade Fire', 'Yeah Yeah Yeahs', 'Foals', 'Kings of Leon', 'The Strokes', 'MGMT', 'Bloc Party'] ["It's Blitz!", 'Oracular Spectacular']
['Time to Pretend', 'Walking on a Dream'] ['Arcade Fire', 'Yeah Yeah Yeahs', 'Foals', 'Kings of Leon', 'The Strokes', 'MGMT', 'Bloc Party'] ["It's Blitz!", 'Oracular Spectacular']
5 7 11
[] ['Radiohead', 'Daft Punk'] ['Fever Ray']
[] ['Radiohead'] []
[] ['The Shins', 'The Rolling Stones', 'Sufjan Stevens', 'Bob Dylan', 'The Beatles'] ['Wincing the Night Away']
[] ['The Shins', 'The Rolling Stones', 'Sufjan Stevens', 'Bob Dylan', 'The Beatles'] ['Wincing the Night Away']
5 14 6
[] ['The Rolling Stones'] []
[] ['Amy Winehouse'] ['Back to Black']
[] [] ['Antidotes']
[] ['The Shins', 'Arcade Fire', 'Radiohead', 'Foals', 'LCD Soundsystem', 'Band of Horses', 'The Strokes', 'Bloc Party'] ['Wincing the Night Away']
[] ['The Shins', 'Arcade Fire', 'Radiohead', 'Foals', 'LCD Sound

In [17]:
def shared_songs(user1, user2):
    song_list = []
    user1_songs, user2_songs = top_50_songs(user1), top_50_songs(user2)
    for song in user1_songs:
        if song in user2_songs:
            song_list.append(song)
    return song_list

def shared_artists(user1, user2):
    artist_list = []
    user1_artists, user2_artists = top_50_artists(user1), top_50_artists(user2)
    for artist in user1_artists:
        if artist in user2_artists:
            artist_list.append(artist)
    return artist_list

def shared_albums(user1, user2):
    album_list = []
    user1_albums, user2_albums = top_50_albums(user1), top_50_albums(user2)
    for album in user1_albums:
        if album in user2_albums:
            album_list.append(album)
    return album_list

def similar_songs(song):
    user_list = []
    for i in range(1, 5000):
        if song in top_50_songs(i):
            user_list.append(i)
    song_list = []
    for i in range(len(user_list)):
        for j in range(i + 1, len(user_list)):
            songs = shared_songs(user_list[i], user_list[j])
            for s in songs:
                if(s not in song_list and s != song):
                    song_list.append(s)
    return song_list

print(shared_songs(5, 7))
## print(similar_songs("Bad Apple!!"))

['Time to Pretend', 'Walking on a Dream']


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def similarity_coefficient(user1, user2):
    songs1, songs2 = top_50_songs(user1), top_50_songs(user2)
    artists1, artists2 = top_50_artists(user1), top_50_artists(user2)
    albums1, albums2 = top_50_albums(user1), top_50_albums(user2)

    # 1. Vectorize the strings
    vectorizer = TfidfVectorizer()
    tfidf_matrix_songs = vectorizer.fit_transform(songs1 + songs2)
    tfidf_matrix_artists = vectorizer.fit_transform(artists1 + artists2)
    tfidf_matrix_albums = vectorizer.fit_transform(albums1 + albums2)

    # Separate the matrices for each list
    tfidf_list_songs1 = tfidf_matrix_songs[:len(songs1)]
    tfidf_list_songs2 = tfidf_matrix_songs[len(songs1):]
    tfidf_list_artists1 = tfidf_matrix_artists[:len(artists1)]
    tfidf_list_artists2 = tfidf_matrix_artists[len(artists1):]
    tfidf_list_albums1 = tfidf_matrix_albums[:len(albums1)]
    tfidf_list_albums2 = tfidf_matrix_albums[len(albums1):]

    # 2. Calculate cosine similarity between elements of the two lists
    # This will result in a matrix where element (i, j) is the cosine similarity
    # between the i-th string in list1 and the j-th string in list2.
    similarity_matrix_songs = cosine_similarity(tfidf_list_songs1, tfidf_list_songs2)
    similarity_matrix_artists = cosine_similarity(tfidf_list_artists1, tfidf_list_artists2)
    similarity_matrix_albums = cosine_similarity(tfidf_list_albums1, tfidf_list_albums2)
    return sum(sum(similarity_matrix_songs)) + sum(sum(similarity_matrix_artists)) + sum(sum(similarity_matrix_albums))

threshold = 0.1

def user_coefficients(user, threshold):
    users = []
    for i in range(1, 100):
        similarity = similarity_coefficient(user, i)
        self_similarity = similarity_coefficient(user, user)
        ratio = similarity / self_similarity
        if(ratio > threshold and ratio != 1):
            users.append(i)
            print(i, ratio)
    return(users)

print(similarity_coefficient(10, 10))

print(user_coefficients(87, 0.12))

183.01922377028035
1 0.12698421756872436
4 0.12393277678008724
5 0.1474723154073351
7 0.15994988392461987
12 0.12367351428508261
14 0.14920493894063427
15 0.14305485150883815
18 0.15472358966069563
19 0.14926512083600615
20 0.1604025175520939
21 0.15778097986946546
22 0.14001571132770083
24 0.17305659401378942
26 0.1766660578622509
27 0.12829631627652013
28 0.15017143529307123
31 0.20783965997323411
32 0.14884055498590953
33 0.1419131019467363
35 0.1266316332741351
41 0.13087525666747765
42 0.13300544347570079
43 0.1643726526367065
44 0.1623569370705195
46 0.14066131054081582
49 0.12328952141180755
50 0.16349707601524752
51 0.13828510083840997
53 0.1414611889445665
54 0.12929198103566575
55 0.13894737740146343
58 0.16942852830732852
59 0.1668706990278331
62 0.1334145226170512
63 0.1395379526462549
66 0.173348614508865
67 0.15721900743617603
68 0.13633789731321805
69 0.12822557287271732
71 0.18543536477208172
72 0.13183556170790067
73 0.14898728051197366
74 0.15490702105959
75 0.1394313

In [147]:
def all_combinations(list):
    temp = []
    for i in range(len(list)):
        for j in range(i + 1, len(list)):
            temp.append([list[i], list[j]])
    return temp

def new_music(user, t):
    current_songs, current_artists, current_albums = top_50_songs(user), top_50_artists(user), top_50_albums(user)
    similar_users = user_coefficients(user, t)
    user_combinations = all_combinations(similar_users)
    temp_songs, temp_artists, temp_albums = [], [], []
    for combination in user_combinations:
        user1, user2 = combination[0], combination[1]
        new_songs, new_artists, new_albums = shared_songs(user1, user2), shared_artists(user1, user2), shared_albums(user1, user2)
        for song in new_songs:
            if(song not in current_songs and song not in temp_songs):
                temp_songs.append(song)
        for artist in new_artists:
            if(artist not in current_artists and artist not in temp_artists):
                temp_artists.append(artist)
        for album in new_albums:
            if(album not in current_albums and album not in temp_albums):
                temp_albums.append(album)
    print("Songs: ", temp_songs, "\n\nArtists: ", temp_artists, "\n\nAlbums: ", temp_albums)

new_music(165, 0.28)

28 0.2910368298012896
55 0.3147495947830928
71 0.2869807459492855
80 0.2916255827634997
Songs:  ['Misery Business', 'All We Know'] 

Artists:  ['The Smiths', 'Interpol', 'Nirvana', 'My Chemical Romance', '30 Seconds to Mars', 'Avenged Sevenfold', 'Evanescence', 'Metallica', 'Mindless Self Indulgence', 'AFI', 'Linkin Park', 'Seether', 'Brand New', 'Incubus', 'Death Cab for Cutie', 'Arctic Monkeys'] 

Albums:  ['Violent Femmes', 'Greatest Hits', 'Three Cheers for Sweet Revenge', 'The Black Parade', 'All We Know Is Falling', 'Avenged Sevenfold', 'City of Evil', 'Louder Now', 'Finding Beauty in Negative Spaces', 'Wolfmother', "Whatever People Say I Am, That's What I'm Not"]


In [23]:
list1 = [1, 2, 3, 4, 5]
list2 = [3, 4, 5, 6, 7]

# Convert lists to sets to leverage set operations for uniqueness
set1 = set(list1)
set2 = set(list2)

# Perform the union operation on the sets
union_set = set1.union(set2)
# Alternatively, use the | operator for union:
# union_set = set1 | set2

# Convert the resulting set back to a list
union_list = list(union_set)

print("Union of the lists:", union_list)

def all_combinations(list):
    temp = []
    for i in range(len(list)):
        for j in range(i + 1, len(list)):
            temp.append([list[i], list[j]])
    return temp

print(all_combinations(list1))

Union of the lists: [1, 2, 3, 4, 5, 6, 7]
[[1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]


In [153]:
def common_elements(list1, list2):
    sum = 0
    for element in list1:
        if element in list2:
            sum += 1
    return sum

def non_common_elements(list1, list2):
    temp = []
    for element in list1:
        if element not in list2:
            temp.append(element)
    return temp

def compare_users(user):
    top_songs, top_artists, top_albums = top_50_songs(user), top_50_artists(user), top_50_albums(user)

    temp_songs, temp_artists, temp_albums = [], [], []

    for i in range(user - 50, user + 50):
        user_songs, user_artists, user_albums = top_50_songs(i), top_50_artists(i), top_50_albums(i)

        metric = common_elements(top_songs, user_songs) + common_elements(top_artists, user_artists) + common_elements(top_albums, user_albums)
        if(metric > 20):
            temp_songs += non_common_elements(user_songs, top_songs)
            temp_artists += non_common_elements(user_artists, top_artists)
            temp_albums += non_common_elements(user_albums, top_albums)
    print(temp_songs)

compare_users(220)

['여자 대통령', '내 여자야', 'Wa$$up', '티켓 (Ticket)', '이름이 뭐예요?', 'Sunshine', '으르렁 (Growl)', '뉴스 (News)', '2MYX', '상남자 (Boy In Luv)', '요즘 너', '갤럭시', 'A', 'BO$$', 'Oh! My God', 'I Got a Boy', 'Something', 'Off to the Races', '늑대와 미녀 (Wolf)', 'NoNoNo', 'Froot', 'Immortal', '나처럼 해봐요', 'Reflection', 'NO MORE DREAM', 'Bad Girl, Good Girl', 'We Are Bulletproof PT.2', '사랑밖에 난 몰라', 'Up & Down', "B형여자 ('Blood Type B Girl')", 'HOT GAME', 'Ice Cream Cake', 'Work', 'Nicotine', '행복 (Happiness)', 'Dark Paradise', 'Na Ponta Ela Fica', '아라리오', '몰라요', 'Bubblegum Bitch', 'Teen Idle', '저주인형', '너 밖에 몰라 (One Way Love)', 'Blue Jeans', 'Lucky Ones', 'How to Be a Heartbreaker', "I'm a Ruin", '두근두근', '블랙팅커벨', '나쁜 기집애', 'You Need to Calm Down', 'august', 'Big Country', 'Lover', 'champagne problems', 'mirrorball', '‘tis the damn season', 'this is me trying', "Nothing's Free", 'illicit affairs', 'Crystalised', 'The Last Man on Earth', 'gold rush', "Don't Shut Me Down", 'Mariners Apartment Complex', 'my tears ricochet', 'C

In [27]:
print("5 and 21: \nSongs: ", shared_songs(5, 21), "\nArtists: ", shared_artists(5, 21), "\nAlbums: ", shared_albums(5, 21))
print("\n5 and 24: \nSongs: ", shared_songs(5, 24), "\nArtists: ", shared_artists(5, 24), "\nAlbums: ", shared_albums(5, 24))
print("\n21 and 24: \nSongs: ", shared_songs(21, 24), "\nArtists: ", shared_artists(21, 24), "\nAlbums: ", shared_albums(21, 24))

5 and 21: 
Songs:  ['My Girls', 'Mansard Roof', 'Chicago', 'Cape Cod Kwassa Kwassa'] 
Artists:  ['The Shins', 'Arcade Fire', 'Sufjan Stevens', 'Vampire Weekend', 'Belle and Sebastian', 'Sigur Rós', 'The Strokes', 'The Beatles', 'MGMT', 'Lady Gaga', 'She & Him', 'Animal Collective'] 
Albums:  ['Funeral', 'Merriweather Post Pavilion', 'Oracular Spectacular', 'Neon Bible', 'Vampire Weekend', 'Back to Black']

5 and 24: 
Songs:  ['Paris'] 
Artists:  ['Bon Iver', 'The Shins', 'Arcade Fire', 'Yeah Yeah Yeahs', 'Regina Spektor', 'Belle and Sebastian', 'Kings of Leon', 'The Strokes', 'The Beatles', 'M.I.A.', 'Beck', 'The Decemberists'] 
Albums:  ['For Emma, Forever Ago', 'Fleet Foxes', 'Lungs', 'Neon Bible']

21 and 24: 
Songs:  [] 
Artists:  ['Belle and Sebastian', 'Arcade Fire', 'The Beatles', 'The Strokes', 'Oasis', 'Arctic Monkeys', 'The Shins', 'Eels'] 
Albums:  ['The Suburbs', 'Neon Bible', 'The Life Pursuit', 'The Bird Of Music']


In [157]:
def common_users(songs = [], artists = [], albums = []):
    temp = []
    for song in songs:
        print(len(df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist()))
        temp.append(df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist())
    for artist in artists:
        temp += (df_artists.loc[df_artists["artist_name"] == artist, "user_id"].tolist())
    for album in albums:
        temp += (df_albums.loc[df_albums["album_name"] == album, "user_id"].tolist())
    intersection_set = set.intersection(*[set(lst) for lst in temp])

    # Convert the resulting set back to a list if needed
    intersection_list = list(intersection_set)
    return intersection_list

test_song = "I'm Dancing in the Show Tonight"
test_song_2 = "Jingle Bells"

test_artist_1 = "Eminem"
test_artist_2 = "Toby Fox"

test_album_1 = "Fleet Foxes"
test_album_2 = "Wincing the Night Away"

print(common_users(songs = [test_song, test_song_2]))

## print("Next\n\n")

## print(common_users(artists = [test_artist_1, test_artist_2], albums = [test_album_1, test_album_2]))

136
207
[339087]


In [291]:
import random

## print(df_tracks.loc[df_tracks["track_name"] == test_song_2, "user_id"].tolist())

## print(df_tracks.loc[(df_tracks["track_name"] == "Lollipop") & (df_tracks["artist_name"] == "Mika")])

def random_choices(songs, N):
    temp = []
    users = []
    for song in songs:
        users += df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist()
    while(len(temp) < N):
        rand = random.sample(users, 2)
        r1, r2 = rand[0], rand[1]
        rand_songs = shared_songs(r1, r2)
        if(len(rand_songs) == 0):
            continue
        rand_song = random.sample(rand_songs, 1)[0]
        if(rand_song not in temp and rand_song not in songs):
            temp.append(rand_song)
    return temp

print(random_choices(["Fly Me To the Moon", "Mack the Knife", "Runaround Sue"], 5))

['Shine', 'Stand by Me', 'Holiday', 'Dream Lover', 'I Want You Back']


In [255]:
lists = [[1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 6, 7]]

# Convert each list to a set and then find the intersection of all sets
intersection_set = set.intersection(*[set(lst) for lst in lists])

# Convert the resulting set back to a list if needed
intersection_list = list(intersection_set)

print(intersection_list)

print(top_50_songs(50))

print(df_tracks.loc[df_tracks["artist_name"] == "DAX"])

all_songs = []

songs = list(set(df_tracks["track_name"].tolist()))
users = 

print(len(songs))

[3, 4]
['Grounds for Divorce', 'Dear Sons and Daughters of Hungry Ghosts', 'Staring At The Sun', 'We Built Another World', "It's a Curse", 'Danse Macabre', 'Home', 'Disco Sheets', 'Lousy Pictures', 'Grass', 'Shine a Light', "The National People's Scare", 'Fog', 'Dashboard', "I'll Believe in Anything", 'Migration', 'Trains to Brazil', 'Float On', 'The Prayer', 'Paper Thin Walls', 'Faithful Guide', 'Did You See the Words', 'Killing Armies', "This Heart's on Fire", 'I Came as a Rat', 'Read My Mind', 'Jaws of Life', 'Listen (Listen, Listen)', 'Sore', 'Dog Days Are Over', 'All Fires', 'Hey Now Now', 'Plea From a Cat Named Virtue', 'DLZ', 'Banshee Beat', 'Assembly Lines', 'Nerves Normal, Breath Normal', 'Fancy Claps', 'A Different City', 'Shut Up I Am Dreaming Of Places Where Lovers Have Wings', 'People Talk', 'Dinner Bells', 'Flesh Canoe', 'Our Swords', '3rd Planet', 'A Long Flight', 'Modern World', 'The Purple Bottle', 'Going to Your Funeral, Pt. 1', 'I Was A Lover']
          user_id  ran